# Hunalign on non-interlaced text on Lotsawa

In [1]:
../hunalign/src/hunalign/hunalign null.dic lotsawa_interlace.bo.stem  lotsawa_interlace.en.stem -noalign


../hunalign/src/hunalign/hunalign -batch lotsawa_interlace.autodict noninterlace.jobs

SyntaxError: invalid syntax (<ipython-input-1-a54612fc473c>, line 1)

In [27]:
import glob
from util import *
bo_files = glob.glob("articles/*.bo.stem")
with open("noninterlace.jobs", 'w') as f:
    for bo_file in bo_files:
        en_file = bo_file.replace("bo.stem", "en.stem")
        align_file = bo_file.replace("bo.stem", "align")
        f.write(bo_file+'\t'+en_file+"\t"+align_file+"\n")


In [28]:
# Combine and convert to text after align

In [42]:
align_files = glob.glob("articles/*.align")

data = []  # source, target, score, source_url, target_url
import re
for file in align_files:
    en_file = file.replace(".align", ".en.stem").replace("articles/", "articles_plain/")
    bo_file = file.replace(".align", ".bo.stem").replace("articles/", "articles_plain/")
    en_url = "https://www.lotsawahouse.org/" + re.match("articles/(.+).align", file)[1].replace("_","/")
    bo_url = get_bo_url(en_url)
    
    with open(en_file, 'r') as f:
        en_lines = f.read().split("\n")
    with open(bo_file, 'r') as f:
        bo_lines = f.read().split("\n")
        
    with open(file, 'r') as f:
        lines = f.readlines()
    ins_bo, ins_en, scores = [], [], []
    for line in lines:
        i_bo, i_en, score = line.strip().split("\t")
        ins_bo.append(int(i_bo))
        ins_en.append(int(i_en))
        scores.append(score)
    for i in range(len(lines)-1):
        if ins_bo[i]==ins_bo[i+1] or ins_en[i]==ins_en[i+1]:
            continue
        en_text = " ".join(en_lines[ins_en[i]:ins_en[i+1]])
        bo_text = "".join(bo_lines[ins_bo[i]:ins_bo[i+1]])
        data.append([bo_text, en_text, bo_url, en_url, scores[i]])

In [43]:
import pandas as pd
df = pd.DataFrame(data,columns=["source", "target", "source_url", "target_url", "alignment_score"])

In [44]:
df

,source,target,source_url,target_url,alignment_score
0,ལོག་ལྟའི་སྨག་འབྱིན་མུན་སེལ་ཉི་མ་བཞུགས།,A Sun to Banish the Darkness of Wrong Views,https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,0.275
1,སྲིད་པའི་དྲྭ་བ་བྲལ་མཛད་ཅིང་། །,"You cut apart the web of conditioned existence,",https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,0.953226
2,ཉེས་བརྒྱའི་ཚངས་ཚིངས་བསྲེག་མཛད་པ། །,"And burn away the thicket of abundant faults,",https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,0.865714
3,བློ་གཏེར་རྡོ་རྗེ་རྣོན་པོ་ཡི། །,"Treasury of Wisdom, Adamantine Sharpness,",https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,0.660829
4,ཞབས་ལ་བཏུད་ནས་འདི་བརྗོད་བྱ། །,"At your feet I bow, and impart this message.",https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,1.355
...,...,...,...,...,...
33267,བདག་ཀྱང་དེང་ནས་ཚེ་རབས་ཐམས་ཅད་དུ། །,"Now and in all my lives to come, may I always ...",https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,1.27071
33268,དྲན་པ་ཡེ་ཤེས་མཆོག་དང་མི་འབྲལ་ཤོག །,"From you, the sublime master Smṛtijñāna.",https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,0.624762
33269,ཅེས་པའང་ས་ཁྱི་ཟླ་ཚེས་ལ་རྨི་ལམ་ཉིང་འཁྲུལ་གྱི་རྐ...,This garland of words arose during the Earth D...,https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,0.279259
33270,མངྒ་ལཾ།།,Maṅgalam.,https://www.lotsawahouse.org/bo/tibetan-master...,https://www.lotsawahouse.org/tibetan-masters/j...,1.26667


In [45]:
df.to_csv("bisent_lotsawa_noninterlace.csv", index=False)

In [46]:
df.sample(30, random_state=33).to_csv("bisent_lotsawa_noninterlace_sample30.csv", index=False)

33272